In [2]:
import sys
sys.path.append('../main/ModelHelpers/cINN/model')
sys.path.append('../main/ModelHelpers/cINN/model/modules')

import dataset
import loader
import model_cINN

import torch

In [5]:
paths_to_PS = ["../from_cloud/simData_68500.bp", "../from_cloud/simData_68500.bp"]
paths_to_radiation = ["../from_cloud/b_radAmplitudes_68500_0_0_0.h5", "../from_cloud/b_radAmplitudes_68500_0_0_0.h5"]

In [6]:
dataset_tr = dataset.PCDataset(items_phase_space=paths_to_PS,
                 items_radiation=paths_to_radiation,
                 num_points=100,
                 num_files=-1,
                 chunk_size=20,
                 normalize=True, a=0., b=1.)

Number of simulations:  2
Number of chunks:  10
Get min/max from phase space data...
PS Minima: 
	 tensor([ 0.0000,  0.0073,  0.0000, -0.5310,  0.0000, -0.1218])
PS Maxima: 
	 tensor([5.7769e+02, 9.7963e-01, 9.9243e-01, 9.1658e-36, 4.1451e+01, 1.3520e-01])
Get min/max from radiation data...
PS Minima: 
	 tensor([-326833.2500, -326833.2500, -326833.2500, -326833.2500, -326833.2500,
        -326833.2500])
PS Maxima: 
	 tensor([310444.7500, 310444.7500, 310444.7500, 310444.7500, 310444.7500,
        310444.7500])


In [7]:
loader_tr = loader.get_loader(dataset_tr, batch_size=1)
for ps, rad in loader_tr:
    print(ps.shape)
    print(rad.shape)
    break

Total number of points:  2
Size of loader:  2
torch.Size([1, 20, 6])
torch.Size([1, 20, 440, 2048, 6])


In [8]:
model_f = model_cINN.PC_NF(dim_condition=440*2048*6,
                 dim_input=6,
                 num_coupling_layers=1,
                 num_linear_layers=1,
                 hidden_size=32,
                 device='cuda',
                 enable_wandb=False)

optimizer = torch.optim.Adam(model_f.trainable_parameters, lr=1e-3, betas=(0.8, 0.9),
                             eps=1e-6, weight_decay=2e-5)
test_pointcloud = '../simulations/data/data_npy/training_data_npy/Track_1.017e-08_2.008e-08_1.778e-03_2.556e-03/Track_1.017e-08_2.008e-08_1.778e-03_2.556e-03-UndMid.npy'

model_f.train_(dataset_tr,
               dataset_tr,
               optimizer,
               epochs=1,
               batch_size=1,
               test_epoch=25,
               test_pointcloud=test_pointcloud, log_plots=None,
               path_to_models='./RESmodels_freia/')

Total number of points:  2
Size of loader:  2
epoch : 1/1,
	loss_avg = 3316.924618244171143,
	loss_z = 3319.889225721359253,
	loss_j = 2.964508330915123
Total number of points:  2
Size of loader:  2
Validation:
	loss_avg = 1070.018920898437500,
	loss_z = 1073.984497070312500,
	loss_j = 3.965520977973938
